In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import ecm
import configparser
from openpnm.topotools import plot_connections as pconn
from openpnm.topotools import plot_coordinates as pcoord
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import openpnm as op
%matplotlib inline

In [27]:
wrk = op.Workspace()
save_root = os.path.join(ecm.OUTPUT_DIR, "spiral")
save_fig = os.path.join(save_root, 'tabs.png'),

In [28]:
def plot_topology(net):
    inner = net["pore.inner"]
    outer = net["pore.outer"]
    fig, ax = plt.subplots(figsize=(10, 10))
    ax = pconn(net, throats=net.throats("throat.neg_cc"), c="blue", ax=ax)
    ax = pconn(net, throats=net.throats("throat.pos_cc"), c="red", ax=ax)
    ax = pcoord(net, pores=net["pore.neg_cc"], c="blue", ax=ax)
    ax = pcoord(net, pores=net["pore.pos_cc"], c="red", ax=ax)
    ax = pcoord(net, pores=net["pore.neg_tab"], c="blue", s=300, ax=ax)
    ax = pcoord(net, pores=net["pore.pos_tab"], c="red", s=300, ax=ax)
    ax = pcoord(net, pores=inner, c="pink", ax=ax)
    ax = pcoord(net, pores=outer, c="yellow", ax=ax)
    ax = pcoord(net, pores=net.pores('free_stream'), c="green", ax=ax)
    ax = pconn(net, throats=net.throats("throat.free_stream"), c="green",
                ax=ax)
    t_sep = net.throats("spm_resistor")
    if len(t_sep) > 0:
        ax = pconn(net, throats=net.throats("spm_resistor"),c="k", ax=ax)
    plt.tight_layout()

In [29]:
def update_tabs(project, config):
    net = project.network
    sec = 'GEOMETRY'
    print(net.labels())
    for key in ['pos_tab', 'neg_tab']:
        print(key, 'before', net.pores(key))
        net['pore.'+key] = False
        tab_list = json.loads(config.get(sec, key))
        Ps = net.pores(key.strip('_tab')+'_cc')
        Np = float(len(Ps)-1)
        for i, elem in enumerate(tab_list):
            relative_position = float(elem)
            int_id = int(np.around(relative_position*Np))
            net['pore.'+key.strip('s')][Ps[int_id]] = True
        print(key, 'after', net.pores(key))

In [30]:
def update_plot(p1, p2, p3, p4, p5, n1, n2, n3, n4, n5, net):
    pos_Ps = net.pores('pos_cc')
    neg_Ps = net.pores('neg_cc')
    pos_ints = [p1, p2, p3, p4, p5]
    neg_ints = [n1, n2, n3, n4, n5]
    pos_tabs = pos_Ps[pos_ints]
    neg_tabs = neg_Ps[neg_ints]
    net['pore.pos_tab'] = False
    net['pore.neg_tab'] = False
    net['pore.pos_tab'][pos_tabs] = True
    net['pore.neg_tab'][neg_tabs] = True
    plot_topology(net)

In [54]:
Npoints = 36
dtheta=int(360/Npoints)
spacing=200e-6
inner_r=1.5e-3
pos_tabs=[0]
neg_tabs=[-1]
tesla_tabs=False
Nlayers=5
length_3d=650e-3

In [53]:
project, arc_edges = ecm.make_spiral_net(
    dtheta=dtheta,
    spacing=spacing,
    inner_r=inner_r,
    pos_tabs=pos_tabs,
    neg_tabs=neg_tabs,
    tesla_tabs=tesla_tabs,
    Nlayers=Nlayers,
    length_3d=length_3d
)
net = project.network
Npcc = net.num_pores('pos_cc')
Nncc = net.num_pores('neg_cc')
print('Num pos', Npcc, 'Num neg', Nncc)

------------------------------------------------------------
CRITICAL   : front and back labels have been switched to obey the right-hand rule 
SOURCE     : openpnm.network.Cubic.__init__ 
TIME STAMP : 2024-02-07 12:27:20,261
------------------------------------------------------------
------------------------------------------------------------
CRITICAL   : front and back labels have been switched to obey the right-hand rule 
SOURCE     : openpnm.network.Cubic.__init__ 
TIME STAMP : 2024-02-07 12:27:20,275
------------------------------------------------------------
------------------------------------------------------------
CRITICAL   : front and back labels have been switched to obey the right-hand rule 
SOURCE     : openpnm.network.Cubic.__init__ 
TIME STAMP : 2024-02-07 12:27:20,279
------------------------------------------------------------


Num pos 180 Num neg 180


In [61]:
init_pos = [0 + (Npoints*i) for i in range(5)]
init_neg = [(Nncc-int(Npoints/2)) - (Npoints*i) for i in range(5)]
print(init_pos, init_neg)

[0, 36, 72, 108, 144] [162, 126, 90, 54, 18]


In [62]:
interact_manual(update_plot,
                p1=widgets.IntSlider(min=0, max=Npcc, step=1, value=init_pos[0]),
                p2=widgets.IntSlider(min=0, max=Npcc, step=1, value=init_pos[1]),
                p3=widgets.IntSlider(min=0, max=Npcc, step=1, value=init_pos[2]),
                p4=widgets.IntSlider(min=0, max=Npcc, step=1, value=init_pos[3]),
                p5=widgets.IntSlider(min=0, max=Npcc, step=1, value=init_pos[4]),
                n1=widgets.IntSlider(min=0, max=Nncc, step=1, value=init_neg[0]),
                n2=widgets.IntSlider(min=0, max=Nncc, step=1, value=init_neg[1]),
                n3=widgets.IntSlider(min=0, max=Nncc, step=1, value=init_neg[2]),
                n4=widgets.IntSlider(min=0, max=Nncc, step=1, value=init_neg[3]),
                n5=widgets.IntSlider(min=0, max=Nncc, step=1, value=init_neg[4]),
                net=fixed(net))

interactive(children=(IntSlider(value=0, description='p1', max=180), IntSlider(value=36, description='p2', max…

<function __main__.update_plot(p1, p2, p3, p4, p5, n1, n2, n3, n4, n5, net)>

In [63]:
neg_cc_lens = net['throat.arc_length'][net['throat.neg_cc']]
cum_len = np.cumsum(neg_cc_lens)
P_neg_tab = np.argmin(cum_len < cum_len[-1]/2)

In [64]:
pos_cc_lens = net['throat.arc_length'][net['throat.pos_cc']]
cum_len = np.cumsum(neg_cc_lens)
P_pos_tab = np.argmin(cum_len < cum_len[-1]/2)

In [65]:
print(P_neg_tab, P_pos_tab)

106 106


In [66]:
np.argwhere(net['pore.pos_tab'])

array([[  1],
       [ 73],
       [145],
       [217],
       [289]], dtype=int64)